In [1]:
%matplotlib inline

In [291]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split 

## Data Reading

### WTeams
---
- **TeamID** - a 4 digit id number, from 3000-3999, uniquely identifying each NCAA® women's team. A school's TeamID does not change from one year to the next, so for instance the Duke women's TeamID is 3181 for all seasons. To avoid possible confusion between the men's data and the women's data, all of the men's team ID's range from 1000-1999, whereas all of the women's team ID's range from 3000-3999.
- **TeamName** - a compact spelling of the team's college name, 16 characters or fewer. There are no commas or double-quotes in the team names, but you will see some characters that are not letters or spaces, e.g., Texas A&M, St Mary's CA, TAM C. Christi, and Bethune-Cookman.

In [11]:
wteams = pd.read_csv("womens-machine-learning-competition-2019/WDataFiles/WTeams.csv")
print(wteams.shape)
wteams.head()

(366, 2)


,TeamID,TeamName
0,3101,Abilene Chr
1,3102,Air Force
2,3103,Akron
3,3104,Alabama
4,3105,Alabama A&M


### WSeasons
---
- **Season** - indicates the year in which the tournament was played. Remember that the current season counts as 2019.
- **DayZero** - tells you the date corresponding to daynum=0 during that season.
- **RegionW, RegionX, Region Y, Region Z** - by convention, the four regions in the final tournament are always named W, X, Y, and Z. Whichever region's name comes first alphabetically, that region will be Region W. And whichever Region plays against Region W in the national semifinals, that will be Region X. For the other two regions, whichever region's name comes first alphabetically, that region will be Region Y, and the other will be Region Z. 

In [4]:
wseasons = pd.read_csv("womens-machine-learning-competition-2019/WDataFiles/WSeasons.csv")
print(wseasons.shape)
wseasons.head()

(22, 6)


,Season,DayZero,RegionW,RegionX,RegionY,RegionZ
0,1998,10/27/1997,East,Midwest,Mideast,West
1,1999,10/26/1998,East,Mideast,Midwest,West
2,2000,11/1/1999,East,Midwest,Mideast,West
3,2001,10/30/2000,East,Midwest,Mideast,West
4,2002,10/29/2001,East,West,Mideast,Midwest


### WNCAATourneySeeds
---
- **Season** - the year that the tournament was played in
- **Seed** - this is a 3-character identifier of the seed, where the first character is either W, X, Y, or Z (identifying the region the team was in) and the next two digits (either 01, 02, ..., 15, or 16) tell you the seed within the region. 

In [214]:
wseeds = pd.read_csv("womens-machine-learning-competition-2019/WDataFiles/WNCAATourneySeeds.csv")
print(wseeds.shape)
wseeds.head()

(1344, 3)


,Season,Seed,TeamID
0,1998,W01,3330
1,1998,W02,3163
2,1998,W03,3112
3,1998,W04,3301
4,1998,W05,3272


### WRegularSeasonCompactResults
---
- **Season** - this is the year of the associated entry in WSeasons.csv
- **DayNum** - this integer always ranges from 0 to 132, and tells you what day the game was played on. It represents an offset from the "DayZero" date in the "WSeasons.csv" file. For example, the first game in the file was DayNum=18. Combined with the fact from the "WSeasons.csv" file that day zero was 10/27/1997 that year, this means the first game was played 18 days later, or 11/14/1997.
- **WTeamID** - this identifies the id number of the team that won the game, as listed in the "WTeams.csv" file. 
- **WScore** - this identifies the number of points scored by the winning team.
- **LTeamID** - this identifies the id number of the team that lost the game.
- **LScore** - this identifies the number of points scored by the losing team. 
- **NumOT** - this indicates the number of overtime periods in the game, an integer 0 or higher.
- **WLoc** - this identifies the "location" of the winning team. If the winning team was the home team, this value will be "H". If the winning team was the visiting team, this value will be "A". If it was played on a neutral court, then this value will be "N".

In [65]:
regular_compact_results = pd.read_csv("womens-machine-learning-competition-2019/WDataFiles/WRegularSeasonCompactResults.csv")
print(regular_compact_results.shape)
regular_compact_results.head()

(101893, 8)


,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1998,18,3104,91,3202,41,H,0
1,1998,18,3163,87,3221,76,H,0
2,1998,18,3222,66,3261,59,H,0
3,1998,18,3307,69,3365,62,H,0
4,1998,18,3349,115,3411,35,H,0


### WNCAATourneyCompactResults
---
This file identifies the game-by-game NCAA® tournament results for all seasons of historical data. The data is formatted exactly like the WRegularSeasonCompactResults data. Each season you will see 63 games listed, since there are no women's play-in games.

In [14]:
tourney_compact_results = pd.read_csv("womens-machine-learning-competition-2019/WDataFiles/WNCAATourneyCompactResults.csv")
print(tourney_compact_results.shape)
tourney_compact_results.head()

(1323, 8)


,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1998,137,3104,94,3422,46,H,0
1,1998,137,3112,75,3365,63,H,0
2,1998,137,3163,93,3193,52,H,0
3,1998,137,3198,59,3266,45,H,0
4,1998,137,3203,74,3208,72,A,0


### WSampleSubmissionStage1
---
During Stage 1, you are asked to make predictions for all possible matchups from the past five NCAA® tournaments (seasons 2014, 2015, 2016, 2017, and 2018). In Stage 2, you will be asked to make predictions for all possible matchups from the current NCAA® tournament (season 2019).

Since there are 64 teams in the tournament, there are `64*63/2=2016` predictions to make for that year, so a Stage 1 submission file will have 2,016*5=10,080 data rows.

ID - this is a 14-character string of the format SSSS_XXXX_YYYY, where SSSS is the four digit season number, XXXX is the four-digit TeamID of the lower-ID team, and YYYY is the four-digit TeamID of the higher-ID team.
Pred - this contains the predicted winning percentage for the first team identified in the ID field, the one represented above by XXXX.
Example #1: You want to make a prediction for Duke (TeamID=3181) against Arizona (TeamID=3112) in the 2005 tournament, with Duke given a 53% chance to win and Arizona given a 47% chance to win. In this case, Arizona has the lower numerical ID so they would be listed first, and the winning percentage would be expressed from Arizona's perspective (47%)

In [10]:
sample_submission = pd.read_csv("womens-machine-learning-competition-2019/WSampleSubmissionStage1.csv")
print(sample_submission.shape)
sample_submission.head()

(10080, 2)


,ID,Pred
0,2014_3103_3107,0.5
1,2014_3103_3113,0.5
2,2014_3103_3119,0.5
3,2014_3103_3124,0.5
4,2014_3103_3140,0.5


### Team Box Scores
---
This section provides game-by-game stats at a team level (free throws attempted, defensive rebounds, turnovers, etc.) for all regular season, conference tournament, and NCAA® tournament games since the 2009-10 season.

- **WFGM** - field goals made (by the winning team)
- **WFGA** - field goals attempted (by the winning team)
- **WFGM3** - three pointers made (by the winning team)
- **WFGA3** - three pointers attempted (by the winning team)
- **WFTM** - free throws made (by the winning team)
- **WFTA** - free throws attempted (by the winning team)
- **WOR** - offensive rebounds (pulled by the winning team)
- **WDR** - defensive rebounds (pulled by the winning team)
- **WAst** - assists (by the winning team)
- **WTO** - turnovers committed (by the winning team)
- **WStl** - steals (accomplished by the winning team)
- **WBlk** - blocks (accomplished by the winning team)
- **WPF** - personal fouls committed (by the winning team)
(and then the same set of stats from the perspective of the losing team: LFGM is the number of field goals made by the losing team, and so on up to LPF).

**Note**: by convention, "field goals made" (either WFGM or LFGM) refers to the total number of fields goals made by a team, a combination of both two-point field goals and three-point field goals. And "three point field goals made" (either WFGM3 or LFGM3) is just the three-point fields goals made, of course. So if you want to know specifically about two-point field goals, you have to subtract one from the other (e.g., WFGM - WFGM3). And the total number of points scored is most simply expressed as 2*FGM + FGM3 + FTM.

In [24]:
regular_detailed = pd.read_csv("womens-machine-learning-competition-2019/WDataFiles/WRegularSeasonDetailedResults.csv")
print(regular_detailed.shape)
regular_detailed.head()

(46342, 34)


,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2010,11,3103,63,3237,49,H,0,23,54,...,13,6,10,11,27,11,23,7,6,19
1,2010,11,3104,73,3399,68,N,0,26,62,...,21,14,27,14,26,7,20,4,2,27
2,2010,11,3110,71,3224,59,A,0,29,62,...,14,19,23,17,23,8,15,6,0,15
3,2010,11,3111,63,3267,58,A,0,27,52,...,26,16,25,22,22,15,11,14,5,14
4,2010,11,3119,74,3447,70,H,1,30,74,...,17,11,21,21,32,12,14,4,2,14


In [25]:
tourney_detailed = pd.read_csv("womens-machine-learning-competition-2019/WDataFiles/WNCAATourneyDetailedResults.csv")
print(tourney_detailed.shape)
tourney_detailed.head()

(567, 34)


,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2010,138,3124,69,3201,55,N,0,28,57,...,34,3,5,17,19,12,18,4,1,18
1,2010,138,3173,67,3395,66,N,0,23,59,...,27,14,15,18,26,8,8,8,6,22
2,2010,138,3181,72,3214,37,H,0,26,57,...,15,3,8,10,21,4,16,6,4,20
3,2010,138,3199,75,3256,61,H,0,25,63,...,20,17,22,16,21,13,16,5,4,24
4,2010,138,3207,62,3265,42,N,0,24,68,...,26,11,17,16,22,9,10,3,4,12


### WCities
---
- **CityID** - a four-digit ID number uniquely identifying a city.
- **City** - the text name of the city.
- **State** - the state abbreviation of the state that the city is in. In a few rare cases, the game location is not inside one of the 50 U.S. states and so other abbreviations are used, for instance Cancun, Mexico has a state abbreviation of MX.

In [22]:
wcities = pd.read_csv("womens-machine-learning-competition-2019/WDataFiles/WCities.csv")
print(wcities.shape)
wcities.head()

(421, 3)


,CityID,City,State
0,4001,Abilene,TX
1,4002,Akron,OH
2,4003,Albany,NY
3,4004,Albuquerque,NM
4,4005,Allentown,PA


### WGameCities
---
- **Season, DayNum, WTeamID, LTeamID** - these four columns are sufficient to uniquely identify each game. Additional data, such as the score of the game and other stats, can be found in the corresponding Compact Results file.
- **CRType** - this can be either Regular or NCAA. If it is Regular, you can find more about the game in the WRegularSeasonCompactResults.csv file. If it is NCAA, you can find more about the game in the WNCAATourneyCompactResults.csv file.
- **CityID** - the ID of the city where the game was played, as specified by the CityID column in the WCities.csv file.

In [3]:
wgame_cities = pd.read_csv("womens-machine-learning-competition-2019/WDataFiles/WGameCities.csv")
print(wgame_cities.shape)
wgame_cities.head()

(46842, 6)


,Season,DayNum,WTeamID,LTeamID,CRType,CityID
0,2010,11,3103,3237,Regular,4002
1,2010,11,3104,3399,Regular,4085
2,2010,11,3110,3224,Regular,4363
3,2010,11,3111,3267,Regular,4158
4,2010,11,3119,3447,Regular,4367


### WTeamSpellings
---
- **TeamNameSpelling** - this is the spelling of the team name. It is always expressed in all lowercase letters - e.g. "ball state" rather than "Ball State" - in order to emphasize that any comparisons should be case-insensitive when matching.
- **TeamID** - this identifies the TeamID for the team that has the alternative spelling (as described in WTeams.csv).

In [8]:
wteam_spellings = pd.read_csv("womens-machine-learning-competition-2019/WDataFiles/WTeamSpellings.csv", engine="python")
print(wteam_spellings.shape)
wteam_spellings.head()

(1141, 2)


,TeamNameSpelling,TeamID
0,a&m-corpus chris,3394
1,a&m-corpus christi,3394
2,abilene chr,3101
3,abilene christian,3101
4,abilene-christian,3101


### WNCAATourneySlots
---
(Matches Scheduler)
- **Slot** - this uniquely identifies one of the tournament games. It is a four-character string, where the first two characters tell you which round the game is (R1, R2, R3, R4, R5, or R6) and the second two characters tell you the expected seed of the favored team. Thus the first row is R1W1, identifying the Round 1 game played in the W bracket, where the favored team is the 1 seed. As a further example, the R2W1 slot indicates the Round 2 game that would have the 1 seed from the W bracket, assuming that all favored teams have won up to that point. The slot names are different for the final two rounds, where R5WX identifies the national semifinal game between the winners of regions W and X, and R5YZ identifies the national semifinal game between the winners of regions Y and Z, and R6CH identifies the championship game.
- **StrongSeed** - this indicates the expected stronger-seeded team that plays in this game. For Round 1 games, a team seed is identified in this column (as listed in the "Seed" column in the WNCAATourneySeeds.csv file), whereas for subsequent games, a slot is identified in this column. In the first record of this file (slot R1W1), we see that seed W01 is the "StrongSeed". Whereas for games from Round 2 or later, rather than a team seed, we will see a "slot" referenced in this column. So in the 33rd record of this file (slot R2W1), it tells us that the winners of slots R1W1 and R1W8 will face each other in Round 2. Of course, in the last few games of the tournament - the national semifinals and finals - it's not really meaningful to talk about a "strong seed" or "weak seed", since you would have #1 seeds favored to face each other, but those games are nevertheless represented in the same format for the sake of consistency.
- **WeakSeed** - this indicates the expected weaker-seeded team that plays in this game, assuming all favored teams have won so far. For Round 1 games, a team seed is identified in this column (as listed in the "Seed" column in the WNCAATourneySeeds.csv file), whereas 

In [9]:
tourney_slots = pd.read_csv("womens-machine-learning-competition-2019/WDataFiles/WNCAATourneySlots.csv")
print(tourney_slots.shape)
tourney_slots.head()

(63, 3)


,Slot,StrongSeed,WeakSeed
0,R1W1,W01,W16
1,R1W2,W02,W15
2,R1W3,W03,W14
3,R1W4,W04,W13
4,R1W5,W05,W12


### Sample Submission
---
Stage 2

In [22]:
submission = pd.read_csv("womens-machine-learning-competition-2019/WSampleSubmissionStage2.csv")
print(submission.shape)
submission.head()

(2016, 2)


,ID,Pred
0,2019_3101_3113,0.5
1,2019_3101_3114,0.5
2,2019_3101_3120,0.5
3,2019_3101_3124,0.5
4,2019_3101_3125,0.5


## Feature Engineering

In [26]:
# regular detail X tourney detail
regular_detailed.columns

Index(['Season', 'DayNum', 'WTeamID', 'WScore', 'LTeamID', 'LScore', 'WLoc',
       'NumOT', 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR',
       'WAst', 'WTO', 'WStl', 'WBlk', 'WPF', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3',
       'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF'],
      dtype='object')

In [266]:
# add match type
regular_detailed["Type"] = "regular"
tourney_detailed["Type"] = "tourney"

# combine data
match_detailed = pd.concat([regular_detailed, tourney_detailed])

# get match id: team1_team2(team1 < team2)
match_detailed["Match"] = match_detailed \
                                .apply(lambda row: "_".join(map(str, sorted([row["WTeamID"], row["LTeamID"]]))), axis=1)

match_detailed["Team1"] = match_detailed["Match"].apply(lambda x: int(x.split("_")[0]))
match_detailed["Team2"] = match_detailed["Match"].apply(lambda x: int(x.split("_")[1]))

match_detailed["Label"] = match_detailed.apply(lambda row: 1 if row["WTeamID"] == row["Team1"] else 0, axis=1)

# WLoc to numeric type
match_detailed["WLoc"] = match_detailed["WLoc"].map({"H": 0, "A": 1, "N": 3})
match_detailed["Type"] = match_detailed["Type"].map({"regular": 0, "tourney": 1})

print(match_detailed.shape)
match_detailed[["Season", "Team1", "Team2", "Match", "Type", "WLoc", "Label"]].head()

(46909, 40)


,Season,Team1,Team2,Match,Type,WLoc,Label
0,2010,3103,3237,3103_3237,0,0,1
1,2010,3104,3399,3104_3399,0,3,1
2,2010,3110,3224,3110_3224,0,1,1
3,2010,3111,3267,3111_3267,0,1,1
4,2010,3119,3447,3119_3447,0,0,1


In [267]:
# test data
test = pd.DataFrame()
test["Season"] = submission["ID"].apply(lambda x: int(x.split("_")[0]))
test["Team1"] = submission["ID"].apply(lambda x: int(x.split("_")[1]))
test["Team2"] = submission["ID"].apply(lambda x: int(x.split("_")[2]))
test["Match"] = test.apply(lambda row: "_".join([str(row["Team1"]), str(row["Team2"])]), axis=1)
test["Type"] = 0
print(test.shape)
test.head()

(2016, 5)


,Season,Team1,Team2,Match,Type
0,2019,3101,3113,3101_3113,0
1,2019,3101,3114,3101_3114,0
2,2019,3101,3120,3101_3120,0
3,2019,3101,3124,3101_3124,0
4,2019,3101,3125,3101_3125,0


In [269]:
# match level feature
stats = ["min", "mean", "max"]
win_feats = ['WScore', 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR', 'WAst', 
             'WTO', 'WStl', 'WBlk', 'WPF']
lose_feats = ['LScore', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3', 'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 
              'LTO', 'LStl', 'LBlk', 'LPF']

win_team_stats = match_detailed[['WTeamID']+win_feat].groupby("WTeamID")\
                .agg( {i: stats for i in win_feat} ).reset_index()

win_team_stats.columns = ["WTeamID"] + ["_".join([i, j]) for i in win_feat for j in stats]

lose_team_stats = match_detailed[['LTeamID']+lose_feat].groupby("LTeamID")\
                .agg( {i: stats for i in lose_feat} ).reset_index()

lose_team_stats.columns = ["LTeamID"] + ["_".join([i, j]) for i in lose_feat for j in stats]

print(win_team_stats.head(3))
print(lose_team_stats.head(3))

   WTeamID  WScore_min  WScore_mean  WScore_max  WFGM_min  WFGM_mean  \
0     3101          53    76.441558          96        17  25.974026   
1     3102          52    67.343750          91        17  23.625000   
2     3103          54    76.718310         118        19  27.422535   

   WFGM_max  WFGA_min  WFGA_mean  WFGA_max   ...     WTO_max  WStl_min  \
0        38        47  58.857143        75   ...          28         3   
1        31        47  57.093750        71   ...          29         1   
2        41        43  61.760563        84   ...          27         1   

   WStl_mean  WStl_max  WBlk_min  WBlk_mean  WBlk_max  WPF_min   WPF_mean  \
0   9.077922        18         0   4.064935        10        8  16.857143   
1   8.781250        15         0   2.437500         9       10  18.062500   
2   7.330986        15         0   2.964789        11        5  16.176056   

   WPF_max  
0       29  
1       28  
2       27  

[3 rows x 43 columns]
   LTeamID  LScore_min  LScore

In [270]:
train = match_detailed[['Season', 'Team1', 'Team2', 'Match', 'Type', 'Label']]
# features for train
train = train.merge(win_team_stats, left_on="Team1", right_on="WTeamID")
train = train.merge(win_team_stats, left_on="Team2", right_on="WTeamID")

train = train.merge(lose_team_stats, left_on="Team1", right_on="LTeamID")
train = train.merge(lose_team_stats, left_on="Team2", right_on="LTeamID")

train.drop(["WTeamID_x", "WTeamID_y", "LTeamID_x", "LTeamID_y"], axis=1, inplace=True)

# features for test
test = test.merge(win_team_stats, left_on="Team1", right_on="WTeamID")
test = test.merge(win_team_stats, left_on="Team2", right_on="WTeamID")

test = test.merge(lose_team_stats, left_on="Team1", right_on="LTeamID")
test = test.merge(lose_team_stats, left_on="Team2", right_on="LTeamID")

test.drop(["WTeamID_x", "WTeamID_y", "LTeamID_x", "LTeamID_y"], axis=1, inplace=True)

In [274]:
print(train.shape)
train[['Season', 'Team1', 'Team2', 'Match', 'Label']].head()

(46909, 174)


,Season,Team1,Team2,Match,Label
0,2010,3103,3237,3103_3237,1
1,2011,3103,3237,3103_3237,1
2,2013,3125,3237,3125_3237,0
3,2014,3125,3237,3125_3237,0
4,2016,3125,3237,3125_3237,0


In [284]:
# get seeds diff
wseeds["region"] = wseeds.apply(lambda row: row["Seed"][0], axis=1)
wseeds["seed_int"] = wseeds.apply(lambda row: int(row["Seed"][1:]), axis=1)

x = train[["Season", "Team1", "Team2"]].copy()
x = x.merge(wseeds[["Season", "TeamID", "seed_int"]], left_on=["Season", "Team1"], right_on=["Season", "TeamID"])
x = x.merge(wseeds[["Season", "TeamID", "seed_int"]], left_on=["Season", "Team2"], right_on=["Season", "TeamID"])
x["seed_diff"] = x["seed_int_x"] - x["seed_int_y"]

x = x[["Season", "Team1", "Team2", "seed_diff"]]
x.drop_duplicates(inplace=True)
print(x.shape)
x.head()

(2429, 4)


,Season,Team1,Team2,seed_diff
0,2016,3125,3151,1
1,2016,3125,3407,-2
2,2016,3333,3407,-13
3,2016,3106,3407,0
4,2016,3125,3326,10


In [286]:
# merge with train and test
train = train.merge(x, on=["Season", "Team1", "Team2"], how="left")
train["seed_diff"].fillna(0, inplace=True)

test = test.merge(x, on=["Season", "Team1", "Team2"], how="left")
test["seed_diff"].fillna(0, inplace=True)

In [292]:
feat_columns = [c for c in train.columns if c not in ['Team1', 'Team2', 'Match', 'Label']]
target_column = "Label"

In [297]:
# training

X_train, X_val, y_train, y_val = train_test_split(train[feat_columns], train[target_column], test_size=0.3)

lgb_train = lgb.Dataset(X_train, y_train, free_raw_data=False)
lgb_test = lgb.Dataset(X_val, y_val, reference=lgb_train,  free_raw_data=False)

params = {"objective": "binary",
          "metric": {"binary_logloss"},
          "boosting_type": "gbdt",
          "learning_rate": 0.02,
          "max_depth": 4,
          "num_leaves": 50,
          "min_data_in_leaf": 20,
          "min_child_samples": 10,
          }

print('start training...')

model = lgb.train(params,
                  lgb_train,
                  num_boost_round=500,
                  valid_sets=lgb_test,
                  early_stopping_rounds=50,
                  categorical_feature=["Season", "Type"],
                  learning_rates=lambda iter: 0.7 * (0.999 ** iter),
                  feature_name=feat_columns)

start training...
[1]	valid_0's binary_logloss: 0.644937
Train until valid scores didn't improve in 50 rounds.
[2]	valid_0's binary_logloss: 0.620722
[3]	valid_0's binary_logloss: 0.60516
[4]	valid_0's binary_logloss: 0.598543
[5]	valid_0's binary_logloss: 0.593403
[6]	valid_0's binary_logloss: 0.588885
[7]	valid_0's binary_logloss: 0.587123
[8]	valid_0's binary_logloss: 0.585189
[9]	valid_0's binary_logloss: 0.582108
[10]	valid_0's binary_logloss: 0.580393
[11]	valid_0's binary_logloss: 0.578345
[12]	valid_0's binary_logloss: 0.576509
[13]	valid_0's binary_logloss: 0.575236
[14]	valid_0's binary_logloss: 0.574379
[15]	valid_0's binary_logloss: 0.573986
[16]	valid_0's binary_logloss: 0.573668
[17]	valid_0's binary_logloss: 0.573902
[18]	valid_0's binary_logloss: 0.573547
[19]	valid_0's binary_logloss: 0.573706
[20]	valid_0's binary_logloss: 0.573929
[21]	valid_0's binary_logloss: 0.573142
[22]	valid_0's binary_logloss: 0.572936
[23]	valid_0's binary_logloss: 0.572665
[24]	valid_0's bin

In [300]:
# prediction
submission["Pred"] = model.predict(test[feat_columns])
submission.to_csv("sub.csv", index=False)

In [94]:
# match level feature
stats = ["min", "mean", "max", "std"]
num_feats = ['WScore', 'LScore', 'WLoc', 'NumOT', 'WFGM', 'WFGA', 
             'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR', 'WAst', 
             'WTO', 'WStl', 'WBlk', 'WPF', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3',
            'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF']

# get aggregation levels
agg_dict = {}
column_names = ["match_label"]
for col in num_feats:
    agg_dict[col] = stats
    column_names.extend(str(col) + "_{}".format(s) for s in stats)

print("agg_dict", agg_dict)
print("column_names", column_names)

match_feature = train.groupby("match_label").agg(agg_dict).reset_index()
match_feature.columns = column_names

match_feature.head()

agg_dict {'WScore': ['min', 'mean', 'max', 'std'], 'LScore': ['min', 'mean', 'max', 'std'], 'WLoc': ['min', 'mean', 'max', 'std'], 'NumOT': ['min', 'mean', 'max', 'std'], 'WFGM': ['min', 'mean', 'max', 'std'], 'WFGA': ['min', 'mean', 'max', 'std'], 'WFGM3': ['min', 'mean', 'max', 'std'], 'WFGA3': ['min', 'mean', 'max', 'std'], 'WFTM': ['min', 'mean', 'max', 'std'], 'WFTA': ['min', 'mean', 'max', 'std'], 'WOR': ['min', 'mean', 'max', 'std'], 'WDR': ['min', 'mean', 'max', 'std'], 'WAst': ['min', 'mean', 'max', 'std'], 'WTO': ['min', 'mean', 'max', 'std'], 'WStl': ['min', 'mean', 'max', 'std'], 'WBlk': ['min', 'mean', 'max', 'std'], 'WPF': ['min', 'mean', 'max', 'std'], 'LFGM': ['min', 'mean', 'max', 'std'], 'LFGA': ['min', 'mean', 'max', 'std'], 'LFGM3': ['min', 'mean', 'max', 'std'], 'LFGA3': ['min', 'mean', 'max', 'std'], 'LFTM': ['min', 'mean', 'max', 'std'], 'LFTA': ['min', 'mean', 'max', 'std'], 'LOR': ['min', 'mean', 'max', 'std'], 'LDR': ['min', 'mean', 'max', 'std'], 'LAst': ['mi

,match_label,WScore_min,WScore_mean,WScore_max,WScore_std,LScore_min,LScore_mean,LScore_max,LScore_std,WLoc_min,...,LStl_max,LStl_std,LBlk_min,LBlk_mean,LBlk_max,LBlk_std,LPF_min,LPF_mean,LPF_max,LPF_std
0,3101_3102,80,80.0,80,NaN,44,44.0,44,NaN,0,...,2,NaN,1,1.0,1,NaN,24,24.0,24,NaN
1,3101_3124,79,79.0,79,NaN,34,34.0,34,NaN,0,...,8,NaN,3,3.0,3,NaN,19,19.0,19,NaN
2,3101_3146,61,71.4,83,9.555103,49,61.8,76,11.144505,0,...,10,2.701851,1,2.8,5,1.483240,18,19.8,22,1.483240
3,3101_3185,83,83.0,83,NaN,59,59.0,59,NaN,3,...,9,NaN,1,1.0,1,NaN,22,22.0,22,NaN
4,3101_3186,72,75.5,79,4.949747,60,66.0,72,8.485281,0,...,12,0.000000,2,4.0,6,2.828427,19,21.0,23,2.828427


In [95]:
# team feature

# get the latest year stats
year = 2017

sub_train = train[train["Season"]==year]

win_feat = ['WScore', 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR', 'WAst', 'WTO', 'WStl', 'WBlk', 'WPF']
lose_feat = ['LScore', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3', 'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF']

win_columns = ["WTeamID"] + [f + "_mean" for f in win_feat]
lose_columns = ["LTeamID"] + [f + "_mean" for f in lose_feat]

win_stats = sub_train.groupby("WTeamID").agg({f: "mean" for f in win_feat}).reset_index()
win_stats.columns = win_columns

lose_stats = sub_train.groupby("LTeamID").agg({f: "mean" for f in lose_feat}).reset_index()
lose_stats.columns = lose_columns

print(win_stats.head())
print(lose_stats.head())

   WTeamID  WScore_mean  WFGM_mean  WFGA_mean  WFGM3_mean  WFGA3_mean  \
0     3101    76.842105  25.894737  59.842105    9.421053   27.157895   
1     3102    61.500000  20.250000  50.000000    5.000000   14.000000   
2     3103    76.285714  26.428571  59.857143    9.571429   23.857143   
3     3104    76.894737  27.157895  62.473684    5.578947   17.684211   
4     3106    69.230769  22.230769  54.076923    4.230769   14.384615   

   WFTM_mean  WFTA_mean   WOR_mean   WDR_mean  WAst_mean   WTO_mean  \
0  15.631579  21.894737  16.947368  26.473684  16.789474  15.736842   
1  16.000000  21.750000  11.500000  22.250000  12.250000  20.750000   
2  13.857143  19.428571  12.428571  28.857143  16.000000  15.857143   
3  17.000000  25.421053  14.789474  31.684211  14.578947  15.842105   
4  20.538462  27.307692  14.769231  26.923077  13.076923  17.846154   

   WStl_mean  WBlk_mean   WPF_mean  
0   7.421053   4.052632  16.157895  
1   9.750000   4.000000  18.750000  
2   7.285714   3.428571

In [84]:
# get seeds

wseeds["region"] = wseeds.apply(lambda row: row["Seed"][0], axis=1)
wseeds["num_seed"] = wseeds.apply(lambda row: int(row["Seed"][1:]), axis=1)

wseeds.head()

,Season,Seed,TeamID,region,num_seed
0,1998,W01,3330,W,1
1,1998,W02,3163,W,2
2,1998,W03,3112,W,3
3,1998,W04,3301,W,4
4,1998,W05,3272,W,5


In [131]:
# combine features
base_feat = ['Season', 'type', 'match_label', 'Team1', 'Team2', 'Label']

train = train[base_feat]
test = test[base_feat]

print("combining match features")
train = train.merge(match_feature, on="match_label", how="left")
test = test.merge(match_feature, on="match_label", how="left")

print("combining team features")
train = train.merge(win_stats, left_on="Team1", right_on="WTeamID", how="left")
train = train.merge(lose_stats, left_on="Team1", right_on="LTeamID", how="left")

test = test.merge(win_stats, left_on="Team1", right_on="WTeamID", how="left")
test = test.merge(lose_stats, left_on="Team1", right_on="LTeamID", how="left")

print("combining seed feature")
train = train.merge(wseeds[["Season", "region", "num_seed", "TeamID"]], left_on=["Season", "Team1"], right_on=["Season", "TeamID"], how="left")
train = train.merge(wseeds[["Season", "region", "num_seed", "TeamID"]], left_on=["Season", "Team2"], right_on=["Season", "TeamID"], how="left")

test = test.merge(wseeds[["Season", "region", "num_seed", "TeamID"]], left_on=["Season", "Team1"], right_on=["Season", "TeamID"], how="left")
test = test.merge(wseeds[["Season", "region", "num_seed", "TeamID"]], left_on=["Season", "Team2"], right_on=["Season", "TeamID"], how="left")

train["seed_diff"] = train["num_seed_x"] - train["num_seed_y"]
test["seed_diff"] = test["num_seed_x"] - test["num_seed_y"]

print(train.columns)

combining match features
combining team features
combining seed feature
Index(['Season', 'type', 'match_label', 'Team1', 'Team2', 'Label',
       'WScore_min', 'WScore_mean_x', 'WScore_max', 'WScore_std',
       ...
       'LStl_mean_y', 'LBlk_mean_y', 'LPF_mean_y', 'region_x', 'num_seed_x',
       'TeamID_x', 'region_y', 'num_seed_y', 'TeamID_y', 'seed_diff'],
      dtype='object', length=163)


In [160]:
feat_X = [c for c in train.columns if c not in ["match_label", "Team1", "Team2", "WTeamID", "LTeamID", "TeamID_x", "TeamID_y", "Label"] ]
feat_y = "Label"
feat_cat = ["Season", "type", "region_x", "region_y"]

# map categorical features
train["Season"] = train["Season"] - 2010
train["type"] = train["type"].map({"regular": 0, "tourney": 1})
train["region_x"] = train["region_x"].map({"W": 0, "X": 1, "Y": 2, "Z": 3})
train["region_y"] = train["region_y"].map({"W": 0, "X": 1, "Y": 2, "Z": 3})

print(feat_X)

['Season', 'type', 'WScore_min', 'WScore_mean_x', 'WScore_max', 'WScore_std', 'LScore_min', 'LScore_mean_x', 'LScore_max', 'LScore_std', 'WLoc_min', 'WLoc_mean', 'WLoc_max', 'WLoc_std', 'NumOT_min', 'NumOT_mean', 'NumOT_max', 'NumOT_std', 'WFGM_min', 'WFGM_mean_x', 'WFGM_max', 'WFGM_std', 'WFGA_min', 'WFGA_mean_x', 'WFGA_max', 'WFGA_std', 'WFGM3_min', 'WFGM3_mean_x', 'WFGM3_max', 'WFGM3_std', 'WFGA3_min', 'WFGA3_mean_x', 'WFGA3_max', 'WFGA3_std', 'WFTM_min', 'WFTM_mean_x', 'WFTM_max', 'WFTM_std', 'WFTA_min', 'WFTA_mean_x', 'WFTA_max', 'WFTA_std', 'WOR_min', 'WOR_mean_x', 'WOR_max', 'WOR_std', 'WDR_min', 'WDR_mean_x', 'WDR_max', 'WDR_std', 'WAst_min', 'WAst_mean_x', 'WAst_max', 'WAst_std', 'WTO_min', 'WTO_mean_x', 'WTO_max', 'WTO_std', 'WStl_min', 'WStl_mean_x', 'WStl_max', 'WStl_std', 'WBlk_min', 'WBlk_mean_x', 'WBlk_max', 'WBlk_std', 'WPF_min', 'WPF_mean_x', 'WPF_max', 'WPF_std', 'LFGM_min', 'LFGM_mean_x', 'LFGM_max', 'LFGM_std', 'LFGA_min', 'LFGA_mean_x', 'LFGA_max', 'LFGA_std', 'LFG

In [167]:
for c in feat_cat:
    train[c] = train[c].astype("category")
    test[c] = test[c].astype("category")

In [172]:
# training

lgb_train = lgb.Dataset(train[feat_X], train[feat_y], free_raw_data=False)
lgb_test = lgb.Dataset(test[feat_X], test[feat_y], reference=lgb_train,  free_raw_data=False)

params = {"objective": "binary",
          "metric": {"binary_logloss"},
          "boosting_type": "gbdt",
          "learning_rate": 0.01,
          "num_leaves": 31,
          "min_data_in_leaf": 10,
          "min_child_samples": 10,
          }

print('start training...')

model = lgb.train(params,
                  lgb_train,
                  num_boost_round=200,
                  valid_sets=lgb_test,
                  early_stopping_rounds=50,
                  categorical_feature=feat_cat,
                  learning_rates=lambda iter: 0.7 * (0.999 ** iter),
                  feature_name=feat_X)

start training...
[1]	valid_0's binary_logloss: 0.604147
Train until valid scores didn't improve in 50 rounds.
[2]	valid_0's binary_logloss: 0.582143
[3]	valid_0's binary_logloss: 0.578267
[4]	valid_0's binary_logloss: 0.575057
[5]	valid_0's binary_logloss: 0.573407
[6]	valid_0's binary_logloss: 0.579112
[7]	valid_0's binary_logloss: 0.581435
[8]	valid_0's binary_logloss: 0.58602
[9]	valid_0's binary_logloss: 0.59833
[10]	valid_0's binary_logloss: 0.596816
[11]	valid_0's binary_logloss: 0.596971
[12]	valid_0's binary_logloss: 0.597355
[13]	valid_0's binary_logloss: 0.598167
[14]	valid_0's binary_logloss: 0.59585
[15]	valid_0's binary_logloss: 0.596914
[16]	valid_0's binary_logloss: 0.640741
[17]	valid_0's binary_logloss: 0.648534
[18]	valid_0's binary_logloss: 0.648321
[19]	valid_0's binary_logloss: 0.647915
[20]	valid_0's binary_logloss: 0.649412
[21]	valid_0's binary_logloss: 0.653348
[22]	valid_0's binary_logloss: 0.654423
[23]	valid_0's binary_logloss: 0.655298
[24]	valid_0's binar